### Exemplo de Modelo Preditivo Para Realizar Previsões De Compras Futuras 
* Olá , sou Maria Carolina e esta é a minha primeira experiência em análise preditiva com o uso do Suporte Vector Machine (SVM).
* Este projeto esta sendo desenvolvido com base em estudos a partir de uma aula em vídeo  

### Definição do problema de Negócio: 
* o primeiro passo é definir o objetivo , o problema que a análise vai buscar resolver 
* Nossa tarefa consistirá em avaliar os atributos que impactam a decisão de um usuário ao comprar produtos online e criar um modelo preditivo para fazer previsões de compras futuras.

### Dataset :
 * Online Shoppers Purchasing Intention Dataset:   
https://archive.ics.uci.edu/ml/datasets/Online+Shoppers+Purchasing+Intention+Dataset  
* O conjunto de dados é composto por valores de atributos de 12.330 sessões online distintas. Esses dados foram compilados assegurando que cada sessão é única a um usuário e distribuída ao longo de um ano, para minimizar viés relacionados a campanhas específicas, datas comemorativas, perfis de usuários ou períodos determinados.   
* Incluído no conjunto, existem 10 atributos quantitativos e 8 qualitativos. O atributo 'Revenue' é designado como a variável dependente, servindo como a etiqueta para classificação ou o que denominamos de variável alvo.  


In [ ]:
# Importando bibliotecas que iremos utilizar...
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
from sklearn import svm
import sklearn
import matplotlib
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore', category=DeprecationWarning)

### Carregando os Dados :

In [ ]:
df_original = pd.read_csv('online_shoppers_intention.csv')
df_original.head()